In [17]:
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [18]:

def load_data(folder_path):
    x_train = np.load(os.path.join(folder_path, 'x_train.npy'))
    y_train = np.load(os.path.join(folder_path, 'y_train.npy'))
    x_test = np.load(os.path.join(folder_path, 'x_test.npy'))
    y_test = np.load(os.path.join(folder_path, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [19]:
x_train, y_train, x_test, y_test = load_data('dataset/')
df = pd.DataFrame(x_train)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,3.800263,-9.336801,1.887547,8.695379,-5.467888,-0.417506,6.437264,6.034322,-6.744333,-4.804074,...,0.422196,0.455706,-0.392614,-0.071493,-0.422519,0.015728,0.371949,-0.510896,-0.185008,0.830435
1,-0.583810,-6.129973,2.024479,6.215942,-6.587338,-2.368612,2.631253,4.848878,-2.983923,-3.354807,...,-0.050931,-0.577802,0.301587,-0.223486,1.039160,0.334080,-0.015529,-0.197926,0.141381,-0.618772
2,6.841292,6.092164,-2.906676,-2.175674,0.328356,-4.625822,-3.320845,-5.158869,1.074492,-0.895219,...,-0.783356,0.296611,0.928290,0.519482,-1.255517,-0.630858,0.514689,1.162088,0.189910,-0.239772
3,-8.539494,-1.932550,1.913704,-0.380637,3.911062,-4.017056,-1.793027,2.514312,-3.903759,-1.914817,...,0.341613,0.033174,-0.170253,0.042089,0.026970,-0.030798,-0.331591,0.258198,-0.094630,0.464437
4,-4.367019,4.663309,-8.983503,-1.639408,2.740054,4.763117,-4.593241,8.509668,2.915853,1.206387,...,-0.357705,0.056675,-0.221779,-0.226112,0.203785,0.390319,0.082990,-0.140710,0.493635,0.381112


В данной лабораторной работе будет практиковаться поиск гиперпараметров. Буду рассмотрены алгоритмы поиска гиперпараметров: grid search, random search.

Помимо поиска гиперпараметров будет рассмотрен алгоритм кросс-валидации, позволяющий получить более достоверную оценку качества модели в условиях недостатка данных.
Хотя в работе предоставлена тестовая выборка, она имеет сугубо теоретический характер (для получения финальной оценки) и на практике как правило недоступна. Поэтому во время подбора гиперпараметров используются лишь `x_train, y_train`. `x_test, y_test` используются лишь для получения финальной оценки, чтобы можно было видеть разницу между разными алгоритмами подбора гиперпараметров (если она будет).

Выберите одну модель из списка: MLPClassifier, SGDClassifier, DecisionTreeClassifier, RandomForestClassifier, SVC.
Для выбранной модели произведите поиск оптимальных гиперпараметров. Требование: поиск должен идти как минимум для двух гиперпараметров.

## 0. Обучение бейзлайн модели для проведения сравнения

In [28]:
# Обучите бейзлайн модель без изменения гиперпараметров.
# Используйте результаты для дальнешейго анализа результатов.

# Напишите ваш код здесь.

from sklearn.neural_network import MLPClassifier

mlpModel = MLPClassifier(activation='identity', max_iter=100)
mlpModel.fit(x_train, y_train);
predict = mlpModel.predict(x_test)

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_score, recall_score, f1_score, log_loss, classification_report, confusion_matrix, RocCurveDisplay

accuracy = accuracy_score(y_test, predict)
f1 = f1_score(y_test, predict, average='weighted')

print('Prediction  - ',predict)
print('True result - ',y_test)
print('Model accuracy - {0}'.format(accuracy))
print('Model f1 - {0}'.format(f1))
print('\n\nClassification report\n\n', classification_report(y_test, predict))

Prediction  -  [0 0 0 1 1 1 6 5 8 7 3 3 4 4 4 5 5 5 0 5 6 7 7 7 8 5 3 8 7 7]
True result -  [0 0 0 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 7 7 7 8 8 8 9 9 9]
Model accuracy - 0.6333333333333333
Model f1 - 0.5590476190476191


Classification report

               precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      1.00      1.00         3
           2       0.00      0.00      0.00         3
           3       0.67      0.67      0.67         3
           4       1.00      1.00      1.00         3
           5       0.50      1.00      0.67         3
           6       0.50      0.33      0.40         3
           7       0.50      1.00      0.67         3
           8       0.33      0.33      0.33         3
           9       0.00      0.00      0.00         3

    accuracy                           0.63        30
   macro avg       0.52      0.63      0.56        30
weighted avg       0.53      0.63      0.56       

## 1. K-Fold Cross-Validation

In [21]:
# Реализуйте фунцию кросс-валидации

# Замечание: x_test, y_test не должны применятся в рамках данной функции.
from sklearn.model_selection import KFold
import statistics as stat

def kfold_cv(model_fn, eval_fn, x: np.ndarray, y: np.ndarray, n_splits=5) -> float:
    """
    Parameters
    ----------
    model_fn : callable
        Функция, что возвращает объект модели.
    eval_fn : _type_
        Функция вида `eval_fn(labels, predictions)`, что возвращает скаляр (значение метрики).
    x : np.ndarray
        Набор признаков (размерность NxD, N - количество экземпляро, D - количество признаков).
    y : np.ndarray
        Набор меток (размерность N)
    n_splits : int, optional
        Количество фолдов, по умолчанию 5.

    Returns
    -------
    float
        Среднее значение метрики по фолдам.
    """
    # Напишите ваш код здесь.
    kf = KFold(n_splits)
    eval = []
    for train_index, test_index in kf.split(x_train):
      X_train, X_test = x[train_index], x[test_index]
      Y_train, Y_test = y[train_index], y[test_index]
      model_fn.fit(X_train, Y_train)
      eval.append(eval_fn(Y_test, model_fn.predict(X_test), average='weighted'))
  
    return stat.mean(eval)

In [22]:
# Протестируйте функцию кросс-валидации.

# Напишите ваш код здесь.


meanValue = kfold_cv(mlpModel, f1_score, x_train, y_train, n_splits=5)
meanValue

0.47361471861471865

## 2. Grid search

In [37]:
# 1. Реализуйте алгоритм поиска гиперпараметров grid search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации.

# Напишите ваш код здесь.

from sklearn.pipeline import Pipeline
import time

result_grid = dict()
parameter_space_grid = {
    'max_iter': [100,120,140, 160, 180, 200,220, 240, 260, 280, 300, 320, 340],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
}

t_begin = time.perf_counter()

for max_iter_param in parameter_space_grid['max_iter']:
    for activation_param in parameter_space_grid['activation']:
        testModel = MLPClassifier(max_iter=max_iter_param, activation=activation_param)
        key = 'Max iterations value - ' + str(max_iter_param) + ' , Activation type - ' + activation_param
        result_grid[key] = kfold_cv(testModel, f1_score, x_train, y_train, n_splits=5)

t_end = time.perf_counter()
print('Execution time:',t_end - t_begin)
print("Optimal values of hyperparameters are following:")
print(max(result_grid, key=result_grid.get))


Execution time: 39.357378100045025
Optimal values of hyperparameters are following:
Max iterations value - 180 , Activation type - tanh


In [42]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test.

# Напишите ваш код здесь.

randomModel = MLPClassifier(max_iter=180, activation='tanh')
randomModel.fit(x_train, y_train);
randomPredict = randomModel.predict(x_test)
randomAccuracy = accuracy_score(y_test, randomPredict)
randomF1 = f1_score(y_test, randomPredict, average='weighted')

print('Prediction  - ',randomPredict)
print('True result - ',y_test)
print('Model accuracy - {0}'.format(randomAccuracy))
print('Model f1 - {0}'.format(randomF1))
print('\n\nClassification report\n\n', classification_report(y_test, randomPredict))


Prediction  -  [0 0 0 1 1 1 3 0 8 3 3 3 4 4 4 5 5 5 0 6 1 7 7 7 2 5 8 5 7 9]
True result -  [0 0 0 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 7 7 7 8 8 8 9 9 9]
Model accuracy - 0.7
Model f1 - 0.6471428571428571


Classification report

               precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       0.75      1.00      0.86         3
           2       0.00      0.00      0.00         3
           3       0.75      1.00      0.86         3
           4       1.00      1.00      1.00         3
           5       0.60      1.00      0.75         3
           6       1.00      0.33      0.50         3
           7       0.75      1.00      0.86         3
           8       0.50      0.33      0.40         3
           9       1.00      0.33      0.50         3

    accuracy                           0.70        30
   macro avg       0.70      0.70      0.65        30
weighted avg       0.70      0.70      0.65        30



## 3. Random search

In [31]:
# 1. Реализуйте алгоритм поиска гиперпараметров random search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации.

# Напишите ваш код здесь.

import random

n_iter = 7
result_random = dict()
parameter_space_random = {
    'max_iter': [100,120,140, 160, 180, 200,220, 240, 260, 280, 300, 320, 340],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
}

t_begin = time.perf_counter()

for i in range(n_iter):
  max_iter_param = parameter_space_random['max_iter'][random.randrange(12)]
  activation_param = parameter_space_random['activation'][random.randrange(3)]
  testModel = MLPClassifier(random_state=random_state_param, max_iter=max_iter_param, activation=activation_param, solver=solver_param)
  key = 'Max iterations value - ' + str(max_iter_param) + ' , Activation type - ' + activation_param
  result_random[key] = kfold_cv(testModel, f1_score, x_train, y_train, n_splits=5)

t_end = time.perf_counter()
print('Execution time:',t_end - t_begin)
print("Optimal values of hyperparameters are following:")
print(max(result_random, key=result_random.get))

Execution time: 6.331007287837565
Optimal values of hyperparameters are following:
Max iterations value - 280 , Activation type - identity


In [32]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test.

# Напишите ваш код здесь.

randomModel = MLPClassifier(max_iter=280, activation='identity')
randomModel.fit(x_train, y_train);
randomPredict = randomModel.predict(x_test)
randomAccuracy = accuracy_score(y_test, randomPredict)
randomF1 = f1_score(y_test, randomPredict, average='weighted')

print('Prediction  - ',randomPredict)
print('True result - ',y_test)
print('Model accuracy - {0}'.format(randomAccuracy))
print('Model f1 - {0}'.format(randomF1))
print('\n\nClassification report\n\n', classification_report(y_test, randomPredict))



Prediction  -  [0 0 0 1 1 1 3 2 3 1 3 3 4 4 4 5 3 5 0 5 4 7 7 7 8 8 8 0 7 9]
True result -  [0 0 0 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 7 7 7 8 8 8 9 9 9]
Model accuracy - 0.7
Model f1 - 0.6488095238095238


Classification report

               precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       0.75      1.00      0.86         3
           2       1.00      0.33      0.50         3
           3       0.40      0.67      0.50         3
           4       0.75      1.00      0.86         3
           5       0.67      0.67      0.67         3
           6       0.00      0.00      0.00         3
           7       0.75      1.00      0.86         3
           8       1.00      1.00      1.00         3
           9       1.00      0.33      0.50         3

    accuracy                           0.70        30
   macro avg       0.69      0.70      0.65        30
weighted avg       0.69      0.70      0.65        30



## 4. Доп. задание (опционально)

### 4.1 Bayesian optimization

Примените байесовскую оптимизацию для поиска гиперпараметров.
В качестве алгоритма используйте `BayesSearchCV` из пакета `scikit-optimize`.

Сложность: почти бесплатный балл.

In [29]:
# 1. Инстанцируйте BayesSearchCV.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.

# Напишите ваш код здесь.

In [30]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test.

# Напишите ваш код здесь.

### 4.2 Tree of Parzen Estimators (TPE) из HyperOpt

Примените TPE из библиотеки hyperopt для поиска гиперпараметров. Вики по HyperOpt: https://github.com/hyperopt/hyperopt/wiki/FMin

Сложность: чтец документаций o(*￣▽￣*)ブ.

In [31]:
def objective(args):
    # Принимает гиперпараметры, инстанцирует модель, обучает её, возвращает значение метрики.
    # Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
    
    # Напишите ваш код здесь.
    pass

In [32]:
# Определите пространство поиска гиперпараметров
space = None

In [33]:
# 1. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 2. Выведите найденные значения гиперпараметров и время работы.

# Напишите ваш код здесь.

In [34]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test.

# Напишите ваш код здесь.